In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow import keras
from google.colab import drive
from datetime import datetime
import matplotlib as mpl
import logging
from PIL import Image
from math import pi
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from itertools import cycle
from matplotlib.patches import Circle, RegularPolygon
from matplotlib.path import Path
from matplotlib.projections.polar import PolarAxes
from matplotlib.projections import register_projection
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D

# Configure matplotlib for professional style
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['axes.grid'] = False
mpl.rcParams['font.size'] = 12
mpl.rcParams['axes.titlesize'] = 16
mpl.rcParams['axes.titleweight'] = 'bold'
mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 10
mpl.rcParams['legend.fontsize'] = 10
sns.set_style("whitegrid")

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('model_testing.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

In [8]:
class ModelTester:
    def __init__(self, model_path, data_dir, img_size=(384, 384), batch_size=32):
        self.model_path = model_path
        self.data_dir = data_dir
        self.img_size = img_size
        self.batch_size = batch_size
        self.model = None
        self.class_names = None
        self.test_ds = None
        self.results_dir = None
        self._setup_directories()

    def _setup_directories(self):


        # Create timestamped results directory
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        self.results_dir = f"/content/drive/MyDrive/Graduation Project/saved_models/Model_Evaluation_Results_Model4/{timestamp}"
        os.makedirs(self.results_dir, exist_ok=True)
        os.makedirs(os.path.join(self.results_dir, "plots"), exist_ok=True)
        os.makedirs(os.path.join(self.results_dir, "metrics"), exist_ok=True)
        os.makedirs(os.path.join(self.results_dir, "misclassified_samples"), exist_ok=True)

        logger.info(f"Results will be saved to: {self.results_dir}")

    def load_model(self):
        """Load the trained model from disk"""
        try:
            logger.info(f"Loading model from {self.model_path}")
            self.model = keras.models.load_model(self.model_path)
            logger.info("Model loaded successfully")

            # Try to plot model architecture
            try:
                from tensorflow.keras.utils import plot_model
                plot_path = os.path.join(self.results_dir, "plots", "model_architecture.png")
                plot_model(
                    self.model,
                    to_file=plot_path,
                    show_shapes=True,
                    show_layer_names=True,
                    rankdir='TB',
                    expand_nested=False,  # Changed to False to avoid Graphviz error
                    dpi=96
                )
                logger.info(f"Model architecture plot saved to {plot_path}")
            except Exception as e:
                logger.warning(f"Could not generate model architecture plot: {str(e)}")
                # Install Graphviz if not available
                logger.info("Attempting to install Graphviz...")
                try:
                    !apt-get install graphviz
                    !pip install pydot
                    from tensorflow.keras.utils import plot_model
                    plot_model(
                        self.model,
                        to_file=plot_path,
                        show_shapes=True,
                        show_layer_names=True,
                        rankdir='TB',
                        expand_nested=False,
                        dpi=96
                    )
                    logger.info(f"Model architecture plot saved to {plot_path}")
                except:
                    logger.warning("Graphviz installation failed. Skipping model plot.")

        except Exception as e:
            logger.error(f"Error loading model: {str(e)}")
            raise

    def create_test_dataset(self):
        """Create a test dataset from the data directory"""
        try:
            logger.info("Creating test dataset")

            # Create test dataset with explicit validation split
            test_ds = keras.utils.image_dataset_from_directory(
                self.data_dir,
                validation_split=0.1,
                subset='validation',
                seed=42,
                image_size=self.img_size,
                batch_size=self.batch_size,
                label_mode='categorical',
                shuffle=True  # Changed to True to ensure better class distribution
            )

            # Get class names from the directory structure
            self.class_names = sorted(os.listdir(self.data_dir))
            logger.info(f"Found {len(self.class_names)} classes: {self.class_names}")

            # Ensure we have samples from all classes
            class_counts = {class_name: 0 for class_name in self.class_names}
            for _, labels in test_ds:
                class_indices = tf.argmax(labels, axis=1)
                for idx in class_indices.numpy():
                    class_counts[self.class_names[idx]] += 1

            logger.info(f"Class distribution in validation set: {class_counts}")

            # Optimize dataset performance
            self.test_ds = test_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

            # Plot class distribution
            self._plot_class_distribution(test_ds)

        except Exception as e:
            logger.error(f"Error creating test dataset: {str(e)}")
            raise

    def _plot_class_distribution(self, dataset):
        """Plot class distribution in the test set"""
        logger.info("Plotting class distribution")

        # Count samples per class
        class_counts = {class_name: 0 for class_name in self.class_names}
        for _, labels in dataset:
            class_indices = tf.argmax(labels, axis=1)
            for idx in class_indices.numpy():
                class_counts[self.class_names[idx]] += 1

        # Create plot with adjusted size for 12 classes
        plt.figure(figsize=(16, 8))
        bars = plt.bar(class_counts.keys(), class_counts.values(), color='#1f77b4')

        # Add value labels
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height}',
                    ha='center', va='bottom', fontsize=9)

        plt.title('Test Set Class Distribution', pad=20)
        plt.xlabel('Class', labelpad=10)
        plt.ylabel('Number of Samples', labelpad=10)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()

        # Save plot
        plot_path = os.path.join(self.results_dir, "plots", "class_distribution.png")
        plt.savefig(plot_path, bbox_inches='tight', dpi=300)
        plt.close()
        logger.info(f"Class distribution plot saved to {plot_path}")

    def evaluate_model(self):
        """Evaluate model performance on test set"""
        if not self.model or not self.test_ds:
            raise ValueError("Model and test dataset must be loaded first")

        logger.info("Evaluating model on test set")

        # Evaluate metrics
        results = self.model.evaluate(self.test_ds, verbose=1)

        # Create a dictionary of metrics
        metrics = dict(zip(self.model.metrics_names, results))

        logger.info("\nTest set evaluation:")
        for name, value in metrics.items():
            logger.info(f"{name}: {value:.4f}")

        # Save metrics to file
        metrics_path = os.path.join(self.results_dir, "metrics", "test_metrics.txt")
        with open(metrics_path, 'w') as f:
            for name, value in metrics.items():
                f.write(f"{name}: {value:.4f}\n")

        logger.info(f"Metrics saved to {metrics_path}")

        return metrics

    def generate_classification_report(self):
        """Generate a detailed classification report"""
        if not self.model or not self.test_ds:
            raise ValueError("Model and test dataset must be loaded first")

        logger.info("Generating classification report")

        # Get true labels and predictions
        y_true = []
        y_pred = []
        y_prob = []
        misclassified_samples = []

        # Ensure we evaluate all batches
        for images, labels in self.test_ds:
            # Get true labels
            y_true.extend(np.argmax(labels.numpy(), axis=1))

            # Get predictions
            preds = self.model.predict(images, verbose=0)
            y_pred.extend(np.argmax(preds, axis=1))
            y_prob.extend(preds)

            # Collect misclassified samples
            true_classes = np.argmax(labels.numpy(), axis=1)
            pred_classes = np.argmax(preds, axis=1)
            for i in range(len(true_classes)):
                if true_classes[i] != pred_classes[i]:
                    misclassified_samples.append({
                        'image': images[i].numpy(),
                        'true_class': true_classes[i],
                        'pred_class': pred_classes[i],
                        'probabilities': preds[i]
                    })

        # Always use all class names, even if some weren't in the validation set
        target_names = self.class_names
        labels = list(range(len(self.class_names)))

        # Generate classification report
        report = classification_report(
            y_true,
            y_pred,
            labels=labels,
            target_names=target_names,
            digits=4,
            output_dict=False
        )

        report_dict = classification_report(
            y_true,
            y_pred,
            labels=labels,
            target_names=target_names,
            digits=4,
            output_dict=True
        )

        logger.info("\nClassification Report:\n" + report)

        # Save report to file
        report_path = os.path.join(self.results_dir, "metrics", "classification_report.txt")
        with open(report_path, 'w') as f:
            f.write(report)

        # Plot precision-recall metrics
        self._plot_precision_recall(report_dict)

        # Plot confusion matrix with all classes
        self._plot_confusion_matrix(y_true, y_pred, labels=labels, target_names=target_names)

        # Generate new plots
        self._generate_radar_plot(report_dict)
        self._plot_roc_curves(y_true, np.array(y_prob))
        self._plot_top_k_accuracy(y_true, np.array(y_prob))
        self._plot_class_confidence_distribution(y_true, np.array(y_prob))

        # Save misclassified samples
        self._save_misclassified_samples(misclassified_samples)

        return report, y_true, y_pred

    def _plot_precision_recall(self, report_dict):
        """Plot precision and recall metrics per class"""
        logger.info("Plotting precision and recall metrics")

        # Prepare data - only include actual classes (skip averages)
        metrics = []
        for class_name in self.class_names:
            if class_name in report_dict:  # Skip 'accuracy', 'macro avg', etc.
                metrics.append({
                    'Class': class_name,
                    'Precision': report_dict[class_name]['precision'],
                    'Recall': report_dict[class_name]['recall'],
                    'F1-Score': report_dict[class_name]['f1-score']
                })

        df = pd.DataFrame(metrics)
        df = df.sort_values('F1-Score', ascending=False)

        # Create plot with adjusted size for 12 classes
        plt.figure(figsize=(18, 8))

        x = np.arange(len(df))
        width = 0.25

        plt.bar(x - width, df['Precision'], width, label='Precision', color='#2ca02c')
        plt.bar(x, df['Recall'], width, label='Recall', color='#1f77b4')
        plt.bar(x + width, df['F1-Score'], width, label='F1-Score', color='#ff7f0e')

        plt.title('Precision, Recall, and F1-Score by Class', pad=20)
        plt.xlabel('Class', labelpad=10)
        plt.ylabel('Score', labelpad=10)
        plt.xticks(x, df['Class'], rotation=45, ha='right')
        plt.ylim(0, 1.1)
        plt.legend(loc='upper right', bbox_to_anchor=(1.15, 1))

        # Add value labels
        for i, (prec, rec, f1) in enumerate(zip(df['Precision'], df['Recall'], df['F1-Score'])):
            plt.text(i - width, prec + 0.02, f"{prec:.2f}", ha='center', fontsize=8)
            plt.text(i, rec + 0.02, f"{rec:.2f}", ha='center', fontsize=8)
            plt.text(i + width, f1 + 0.02, f"{f1:.2f}", ha='center', fontsize=8)

        plt.tight_layout()

        # Save plot
        plot_path = os.path.join(self.results_dir, "plots", "precision_recall.png")
        plt.savefig(plot_path, bbox_inches='tight', dpi=300)
        plt.close()
        logger.info(f"Precision-recall plot saved to {plot_path}")

    def _plot_confusion_matrix(self, y_true, y_pred, normalize=True, cmap='Blues', labels=None, target_names=None):
        """Plot a professional confusion matrix with all classes"""
        logger.info("Generating confusion matrix")

        if labels is None:
            labels = list(range(len(self.class_names)))
        if target_names is None:
            target_names = self.class_names

        # Compute confusion matrix using all specified labels
        cm = confusion_matrix(y_true, y_pred, labels=labels)

        # Fill in zeros for any classes not present in the validation set
        if normalize:
            with np.errstate(divide='ignore', invalid='ignore'):
                cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
                cm[np.isnan(cm)] = 0  # Set NaNs to 0 for classes with no true samples
            fmt = '.2f'
            title = 'Normalized Confusion Matrix'
        else:
            fmt = 'd'
            title = 'Confusion Matrix (Counts)'

        # Create figure with adjusted size
        fig_size = (12 + len(target_names) * 0.5, 10 + len(target_names) * 0.5)
        fig, ax = plt.subplots(figsize=fig_size)

        # Create heatmap
        sns.heatmap(cm, annot=True, fmt=fmt, cmap=cmap, ax=ax,
                    cbar_kws={'label': 'Normalized Count' if normalize else 'Count'},
                    xticklabels=target_names,
                    yticklabels=target_names)

        # Add title and labels
        ax.set_title(title, fontsize=16, pad=20)
        ax.set_xlabel('Predicted Label', fontsize=14, labelpad=10)
        ax.set_ylabel('True Label', fontsize=14, labelpad=10)

        # Rotate tick labels
        plt.setp(ax.get_xticklabels(), rotation=45, ha='right', fontsize=10)
        plt.setp(ax.get_yticklabels(), rotation=0, fontsize=10)

        plt.tight_layout()

        # Save figure
        plot_path = os.path.join(self.results_dir, "plots", "confusion_matrix.png")
        plt.savefig(plot_path, bbox_inches='tight', dpi=300)
        plt.close()

        logger.info(f"Confusion matrix saved to {plot_path}")

    def _save_misclassified_samples(self, misclassified_samples, num_samples=24):
        """Save misclassified samples with their probabilities"""
        if not misclassified_samples:
            logger.info("No misclassified samples found")
            return

        logger.info(f"Saving {min(num_samples, len(misclassified_samples))} misclassified samples")

        # Select random samples if we have too many
        if len(misclassified_samples) > num_samples:
            misclassified_samples = np.random.choice(misclassified_samples, size=num_samples, replace=False)

        # Create figure
        plt.figure(figsize=(20, 20))
        n_cols = 4
        n_rows = int(np.ceil(len(misclassified_samples) / n_cols))

        for i, sample in enumerate(misclassified_samples):
            plt.subplot(n_rows, n_cols, i+1)

            # Display image
            img = (sample['image'] - sample['image'].min()) / (sample['image'].max() - sample['image'].min())
            plt.imshow(img)

            # Get top 3 predicted classes
            top3_idx = np.argsort(sample['probabilities'])[-3:][::-1]
            top3_classes = [self.class_names[idx] for idx in top3_idx]
            top3_probs = [sample['probabilities'][idx] for idx in top3_idx]

            # Create prediction info text
            pred_text = "\n".join([f"{cls}: {prob:.2f}" for cls, prob in zip(top3_classes, top3_probs)])

            # Set title with color coding
            title = f"True: {self.class_names[sample['true_class']]}\nPred: {self.class_names[sample['pred_class']]}\n\n{pred_text}"
            color = 'green' if sample['true_class'] == sample['pred_class'] else 'red'
            plt.title(title, color=color, fontsize=8, pad=4)

            plt.axis('off')

        plt.suptitle('Misclassified Samples with Prediction Probabilities (Top 3)', fontsize=16, y=1.02)
        plt.tight_layout()

        # Save figure
        plot_path = os.path.join(self.results_dir, "plots", "misclassified_samples.png")
        plt.savefig(plot_path, bbox_inches='tight', dpi=150)
        plt.close()

        # Save individual misclassified samples
        for i, sample in enumerate(misclassified_samples[:24]):  # Limit to 24 to avoid too many files
            img = (sample['image'] - sample['image'].min()) / (sample['image'].max() - sample['image'].min())
            img_path = os.path.join(self.results_dir, "misclassified_samples", f"sample_{i}.png")
            plt.imsave(img_path, img)

        logger.info(f"Misclassified samples saved to {plot_path}")

    def run_full_evaluation(self):
        """Run the complete evaluation pipeline"""
        try:
            # Load model and data
            self.load_model()
            self.create_test_dataset()

            # Evaluate metrics
            metrics = self.evaluate_model()

            # Generate reports and plots
            report, y_true, y_pred = self.generate_classification_report()

            # Save results to file
            results_path = os.path.join(self.results_dir, "evaluation_results.txt")
            with open(results_path, "w") as f:
                f.write("Model Evaluation Results\n")
                f.write("="*50 + "\n\n")
                f.write(f"Evaluation Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
                f.write(f"Number of Classes: {len(self.class_names)}\n")
                f.write(f"Classes: {', '.join(self.class_names)}\n\n")
                f.write("Metrics:\n")
                for name, value in metrics.items():
                    f.write(f"{name}: {value:.4f}\n")
                f.write("\nClassification Report:\n")
                f.write(report)

            logger.info(f"Full evaluation completed. Results saved to {self.results_dir}")

            # Create a summary report
            self._create_summary_report(metrics, report)

        except Exception as e:
            logger.error(f"Evaluation failed: {str(e)}")
            raise

    def _create_summary_report(self, metrics, report):
        """Create a visual summary report of the evaluation"""
        logger.info("Creating summary report")

        # Create figure
        fig = plt.figure(figsize=(18, 14), facecolor='white')

        # Add title
        fig.suptitle("Model Evaluation Summary", fontsize=18, fontweight='bold', y=0.97)

        # Create grid for subplots
        gs = fig.add_gridspec(2, 2, hspace=0.3, wspace=0.2)

        # Plot 1: Main metrics
        ax1 = fig.add_subplot(gs[0, 0])
        main_metrics = {k: v for k, v in metrics.items() if k in ['loss', 'accuracy', 'precision', 'recall']}
        colors = ['#d62728', '#2ca02c', '#9467bd', '#ff7f0e']
        ax1.bar(main_metrics.keys(), main_metrics.values(), color=colors)
        ax1.set_title('Key Metrics', fontsize=14)
        ax1.set_ylim(0, 1.1)
        for i, v in enumerate(main_metrics.values()):
            ax1.text(i, v + 0.02, f"{v:.4f}", ha='center')

        # Plot 2: Confusion matrix thumbnail
        ax2 = fig.add_subplot(gs[0, 1])
        cm_path = os.path.join(self.results_dir, "plots", "confusion_matrix.png")
        if os.path.exists(cm_path):
            cm_img = plt.imread(cm_path)
            ax2.imshow(cm_img)
            ax2.axis('off')
            ax2.set_title('Confusion Matrix', fontsize=14)
        else:
            ax2.text(0.5, 0.5, "Confusion Matrix\nNot Available", ha='center', va='center')
            ax2.axis('off')

        # Plot 3: Precision-Recall thumbnail
        ax3 = fig.add_subplot(gs[1, 0])
        pr_path = os.path.join(self.results_dir, "plots", "precision_recall.png")
        if os.path.exists(pr_path):
            pr_img = plt.imread(pr_path)
            ax3.imshow(pr_img)
            ax3.axis('off')
            ax3.set_title('Precision-Recall Metrics', fontsize=14)
        else:
            ax3.text(0.5, 0.5, "Precision-Recall Plot\nNot Available", ha='center', va='center')
            ax3.axis('off')

        # Plot 4: Class distribution thumbnail
        ax4 = fig.add_subplot(gs[1, 1])
        cd_path = os.path.join(self.results_dir, "plots", "class_distribution.png")
        if os.path.exists(cd_path):
            cd_img = plt.imread(cd_path)
            ax4.imshow(cd_img)
            ax4.axis('off')
            ax4.set_title('Class Distribution', fontsize=14)
        else:
            ax4.text(0.5, 0.5, "Class Distribution\nNot Available", ha='center', va='center')
            ax4.axis('off')

        # Add timestamp and class info
        fig.text(0.5, 0.02,
                f"Generated on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} | Classes: {', '.join(self.class_names)}",
                ha='center', fontsize=10)

        # Save summary report
        summary_path = os.path.join(self.results_dir, "summary_report.png")
        plt.savefig(summary_path, bbox_inches='tight', dpi=150)
        plt.close()

        logger.info(f"Summary report saved to {summary_path}")

        plt.axis('off')

        plt.suptitle('Misclassified Samples with Prediction Probabilities (Top 3)', fontsize=16, y=1.02)
        plt.tight_layout()

        # Save figure
        plot_path = os.path.join(self.results_dir, "plots", "misclassified_samples.png")
        plt.savefig(plot_path, bbox_inches='tight', dpi=150)
        plt.close()

        # Save individual misclassified samples
        for i, sample in enumerate(misclassified_samples[:24]):  # Limit to 24 to avoid too many files
            img = (sample['image'] - sample['image'].min()) / (sample['image'].max() - sample['image'].min())
            img_path = os.path.join(self.results_dir, "misclassified_samples", f"sample_{i}.png")
            plt.imsave(img_path, img)

        logger.info(f"Misclassified samples saved to {plot_path}")
    def _generate_radar_plot(self, report_dict):
        """Generate a radar plot for model performance metrics"""
        logger.info("Generating radar plot of performance metrics")

        try:
            # Prepare data for radar plot
            metrics = ['precision', 'recall', 'f1-score']
            classes = [cls for cls in self.class_names if cls in report_dict]

            # Extract values for each class
            data = []
            for cls in classes:
                class_data = []
                for metric in metrics:
                    class_data.append(report_dict[cls][metric])
                data.append(class_data)

            # Calculate averages
            avg_values = [np.mean([data[i][j] for i in range(len(classes))])
                        for j in range(len(metrics))]
            data.append(avg_values)
            classes.append('Average')

            # Create radar plot
            fig = plt.figure(figsize=(10, 10))
            ax = self._radar_factory(len(metrics), frame='polygon')

            # Plot each class
            colors = plt.cm.viridis(np.linspace(0, 1, len(classes)))
            for d, color, label in zip(data, colors, classes):
                ax.plot(angles, d, color=color, linewidth=2, label=label)
                ax.fill(angles, d, color=color, alpha=0.25)

            # Add legend and title
            ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
            plt.title('Class Performance Radar Plot', size=16, pad=20)

            # Save plot
            plot_path = os.path.join(self.results_dir, "plots", "radar_plot.png")
            plt.savefig(plot_path, bbox_inches='tight', dpi=300)
            plt.close()
            logger.info(f"Radar plot saved to {plot_path}")

        except Exception as e:
            logger.warning(f"Could not generate radar plot: {str(e)}")
    def _radar_factory(self, num_vars, frame='circle'):
        """Create a radar chart with num_vars axes"""
        theta = np.linspace(0, 2*np.pi, num_vars, endpoint=False)

        class RadarAxes(PolarAxes):
            name = 'radar'
            def __init__(self, *args, **kwargs):
                super().__init__(*args, **kwargs)
                self.set_theta_zero_location('N')

            def fill(self, *args, closed=True, **kwargs):
                return super().fill(closed=closed, *args, **kwargs)

            def plot(self, *args, **kwargs):
                lines = super().plot(*args, **kwargs)
                for line in lines:
                    self._close_line(line)

            def _close_line(self, line):
                x, y = line.get_data()
                if x[0] != x[-1]:
                    x = np.append(x, x[0])
                    y = np.append(y, y[0])
                    line.set_data(x, y)

            def set_varlabels(self, labels):
                self.set_thetagrids(np.degrees(theta), labels)

            def _gen_axes_patch(self):
                if frame == 'circle':
                    return Circle((0.5, 0.5), 0.5)
                elif frame == 'polygon':
                    return RegularPolygon((0.5, 0.5), num_vars,
                                        radius=0.5, edgecolor="k")
                else:
                    raise ValueError("Unknown value for 'frame': %s" % frame)

            def _gen_axes_spines(self):
                if frame == 'circle':
                    return super()._gen_axes_spines()
                elif frame == 'polygon':
                    spine = Spine(axes=self,
                                spine_type='circle',
                                path=Path.unit_regular_polygon(num_vars))
                    spine.set_transform(Affine2D().scale(0.5).translate(0.5, 0.5))
                    return {'polar': spine}
                else:
                    raise ValueError("Unknown value for 'frame': %s" % frame)

        register_projection(RadarAxes)
        return plt.subplot(111, projection='radar')

    def _plot_roc_curves(self, y_true, y_prob):
        """Plot ROC curves for multiclass classification"""
        logger.info("Generating ROC curves")

        try:
            # Binarize the output
            y_true_bin = label_binarize(y_true, classes=list(range(len(self.class_names))))
            n_classes = y_true_bin.shape[1]

            # Compute ROC curve and ROC area for each class
            fpr = dict()
            tpr = dict()
            roc_auc = dict()
            for i in range(n_classes):
                fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_prob[:, i])
                roc_auc[i] = auc(fpr[i], tpr[i])

            # Compute micro-average ROC curve and ROC area
            fpr["micro"], tpr["micro"], _ = roc_curve(y_true_bin.ravel(), y_prob.ravel())
            roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

            # First aggregate all false positive rates
            all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

            # Then interpolate all ROC curves at these points
            mean_tpr = np.zeros_like(all_fpr)
            for i in range(n_classes):
                mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

            # Finally average it and compute AUC
            mean_tpr /= n_classes

            fpr["macro"] = all_fpr
            tpr["macro"] = mean_tpr
            roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

            # Plot all ROC curves
            plt.figure(figsize=(10, 8))
            colors = cycle(['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
                          '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
                          '#bcbd22', '#17becf', '#1a55FF', '#FF33CC'])

            for i, color in zip(range(n_classes), colors):
                plt.plot(fpr[i], tpr[i], color=color, lw=2,
                        label='ROC curve of class {0} (AUC = {1:0.2f})'
                        ''.format(self.class_names[i], roc_auc[i]))

            plt.plot([0, 1], [0, 1], 'k--', lw=2)
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel('False Positive Rate', fontsize=12)
            plt.ylabel('True Positive Rate', fontsize=12)
            plt.title('Receiver Operating Characteristic (ROC) Curves', fontsize=16, pad=20)
            plt.legend(loc="lower right", bbox_to_anchor=(1.4, 0), fontsize=10)
            plt.grid(True, alpha=0.3)

            # Save plot
            plot_path = os.path.join(self.results_dir, "plots", "roc_curves.png")
            plt.savefig(plot_path, bbox_inches='tight', dpi=300)
            plt.close()
            logger.info(f"ROC curves plot saved to {plot_path}")

        except Exception as e:
            logger.warning(f"Could not generate ROC curves: {str(e)}")

    def _plot_top_k_accuracy(self, y_true, y_prob, max_k=5):
        """Plot top-k accuracy for different values of k"""
        logger.info("Generating top-k accuracy plot")

        try:
            # Calculate top-k accuracy for different k values
            top_k_acc = []
            for k in range(1, max_k + 1):
                correct = 0
                for i in range(len(y_true)):
                    top_k_pred = np.argsort(y_prob[i])[-k:]
                    if y_true[i] in top_k_pred:
                        correct += 1
                top_k_acc.append(correct / len(y_true))

            # Create plot
            plt.figure(figsize=(10, 6))
            plt.plot(range(1, max_k + 1), top_k_acc, marker='o',
                    color='#1f77b4', linewidth=2, markersize=8)

            # Add value labels
            for k, acc in zip(range(1, max_k + 1), top_k_acc):
                plt.text(k, acc + 0.01, f"{acc:.3f}", ha='center', fontsize=10)

            plt.title('Top-k Classification Accuracy', fontsize=16, pad=20)
            plt.xlabel('k (number of top predictions considered)', fontsize=12)
            plt.ylabel('Accuracy', fontsize=12)
            plt.xticks(range(1, max_k + 1))
            plt.ylim(0, 1.1)
            plt.grid(True, alpha=0.3)

            # Save plot
            plot_path = os.path.join(self.results_dir, "plots", "top_k_accuracy.png")
            plt.savefig(plot_path, bbox_inches='tight', dpi=300)
            plt.close()
            logger.info(f"Top-k accuracy plot saved to {plot_path}")

        except Exception as e:
            logger.warning(f"Could not generate top-k accuracy plot: {str(e)}")

    def _plot_class_confidence_distribution(self, y_true, y_prob):
        """Plot distribution of prediction confidence by class"""
        logger.info("Generating class confidence distribution plot")

        try:
            # Prepare data
            confidences = []
            classes = []
            for true_class, probs in zip(y_true, y_prob):
                confidences.append(probs[true_class])
                classes.append(self.class_names[true_class])

            # Create DataFrame for easier plotting
            df = pd.DataFrame({'Class': classes, 'Confidence': confidences})

            # Create plot
            plt.figure(figsize=(12, 8))
            sns.boxplot(x='Class', y='Confidence', data=df, palette='viridis')
            plt.title('Distribution of Prediction Confidence by Class', fontsize=16, pad=20)
            plt.xlabel('Class', fontsize=12)
            plt.ylabel('Confidence', fontsize=12)
            plt.xticks(rotation=45, ha='right')
            plt.ylim(0, 1.1)
            plt.grid(True, alpha=0.3)

            # Save plot
            plot_path = os.path.join(self.results_dir, "plots", "confidence_distribution.png")
            plt.savefig(plot_path, bbox_inches='tight', dpi=300)
            plt.close()
            logger.info(f"Confidence distribution plot saved to {plot_path}")

        except Exception as e:
            logger.warning(f"Could not generate confidence distribution plot: {str(e)}")


    def run_full_evaluation(self):
        """Run the complete evaluation pipeline"""
        try:
            # Load model and data
            self.load_model()
            self.create_test_dataset()

            # Evaluate metrics
            metrics = self.evaluate_model()

            # Generate reports and plots
            report, y_true, y_pred = self.generate_classification_report()

            # Save results to file
            results_path = os.path.join(self.results_dir, "evaluation_results.txt")
            with open(results_path, "w") as f:
                f.write("Model Evaluation Results\n")
                f.write("="*50 + "\n\n")
                f.write(f"Evaluation Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
                f.write(f"Number of Classes: {len(self.class_names)}\n")
                f.write(f"Classes: {', '.join(self.class_names)}\n\n")
                f.write("Metrics:\n")
                for name, value in metrics.items():
                    f.write(f"{name}: {value:.4f}\n")
                f.write("\nClassification Report:\n")
                f.write(report)

            logger.info(f"Full evaluation completed. Results saved to {self.results_dir}")

            # Create a summary report
            self._create_summary_report(metrics, report)

        except Exception as e:
            logger.error(f"Evaluation failed: {str(e)}")
            raise

    def _create_summary_report(self, metrics, report):
        """Create a visual summary report of the evaluation"""
        logger.info("Creating summary report")

        # Create figure with more subplots
        fig = plt.figure(figsize=(20, 24), facecolor='white')

        # Add title
        fig.suptitle("Model Evaluation Summary", fontsize=18, fontweight='bold', y=0.98)

        # Create grid for subplots (now 3 rows, 3 columns)
        gs = fig.add_gridspec(3, 3, hspace=0.4, wspace=0.3)

        # Plot 1: Main metrics
        ax1 = fig.add_subplot(gs[0, 0])
        main_metrics = {k: v for k, v in metrics.items() if k in ['loss', 'accuracy', 'precision', 'recall']}
        colors = ['#d62728', '#2ca02c', '#9467bd', '#ff7f0e']
        ax1.bar(main_metrics.keys(), main_metrics.values(), color=colors)
        ax1.set_title('Key Metrics', fontsize=14)
        ax1.set_ylim(0, 1.1)
        for i, v in enumerate(main_metrics.values()):
            ax1.text(i, v + 0.02, f"{v:.4f}", ha='center')

        # Plot 2: Confusion matrix thumbnail
        ax2 = fig.add_subplot(gs[0, 1])
        cm_path = os.path.join(self.results_dir, "plots", "confusion_matrix.png")
        if os.path.exists(cm_path):
            cm_img = plt.imread(cm_path)
            ax2.imshow(cm_img)
            ax2.axis('off')
            ax2.set_title('Confusion Matrix', fontsize=14)
        else:
            ax2.text(0.5, 0.5, "Confusion Matrix\nNot Available", ha='center', va='center')
            ax2.axis('off')

        # Plot 3: Precision-Recall thumbnail
        ax3 = fig.add_subplot(gs[0, 2])
        pr_path = os.path.join(self.results_dir, "plots", "precision_recall.png")
        if os.path.exists(pr_path):
            pr_img = plt.imread(pr_path)
            ax3.imshow(pr_img)
            ax3.axis('off')
            ax3.set_title('Precision-Recall Metrics', fontsize=14)
        else:
            ax3.text(0.5, 0.5, "Precision-Recall Plot\nNot Available", ha='center', va='center')
            ax3.axis('off')

        # Plot 4: Radar plot thumbnail
        ax4 = fig.add_subplot(gs[1, 0])
        radar_path = os.path.join(self.results_dir, "plots", "radar_plot.png")
        if os.path.exists(radar_path):
            radar_img = plt.imread(radar_path)
            ax4.imshow(radar_img)
            ax4.axis('off')
            ax4.set_title('Performance Radar Plot', fontsize=14)
        else:
            ax4.text(0.5, 0.5, "Radar Plot\nNot Available", ha='center', va='center')
            ax4.axis('off')

        # Plot 5: ROC curves thumbnail
        ax5 = fig.add_subplot(gs[1, 1])
        roc_path = os.path.join(self.results_dir, "plots", "roc_curves.png")
        if os.path.exists(roc_path):
            roc_img = plt.imread(roc_path)
            ax5.imshow(roc_img)
            ax5.axis('off')
            ax5.set_title('ROC Curves', fontsize=14)
        else:
            ax5.text(0.5, 0.5, "ROC Curves\nNot Available", ha='center', va='center')
            ax5.axis('off')

        # Plot 6: Top-k accuracy thumbnail
        ax6 = fig.add_subplot(gs[1, 2])
        topk_path = os.path.join(self.results_dir, "plots", "top_k_accuracy.png")
        if os.path.exists(topk_path):
            topk_img = plt.imread(topk_path)
            ax6.imshow(topk_img)
            ax6.axis('off')
            ax6.set_title('Top-k Accuracy', fontsize=14)
        else:
            ax6.text(0.5, 0.5, "Top-k Accuracy\nNot Available", ha='center', va='center')
            ax6.axis('off')

        # Plot 7: Confidence distribution thumbnail
        ax7 = fig.add_subplot(gs[2, 0])
        conf_path = os.path.join(self.results_dir, "plots", "confidence_distribution.png")
        if os.path.exists(conf_path):
            conf_img = plt.imread(conf_path)
            ax7.imshow(conf_img)
            ax7.axis('off')
            ax7.set_title('Confidence Distribution', fontsize=14)
        else:
            ax7.text(0.5, 0.5, "Confidence Distribution\nNot Available", ha='center', va='center')
            ax7.axis('off')

        # Plot 8: Class distribution thumbnail
        ax8 = fig.add_subplot(gs[2, 1])
        cd_path = os.path.join(self.results_dir, "plots", "class_distribution.png")
        if os.path.exists(cd_path):
            cd_img = plt.imread(cd_path)
            ax8.imshow(cd_img)
            ax8.axis('off')
            ax8.set_title('Class Distribution', fontsize=14)
        else:
            ax8.text(0.5, 0.5, "Class Distribution\nNot Available", ha='center', va='center')
            ax8.axis('off')

        # Plot 9: Misclassified samples thumbnail
        ax9 = fig.add_subplot(gs[2, 2])
        mis_path = os.path.join(self.results_dir, "plots", "misclassified_samples.png")
        if os.path.exists(mis_path):
            mis_img = plt.imread(mis_path)
            ax9.imshow(mis_img)
            ax9.axis('off')
            ax9.set_title('Misclassified Samples', fontsize=14)
        else:
            ax9.text(0.5, 0.5, "Misclassified Samples\nNot Available", ha='center', va='center')
            ax9.axis('off')

        # Add timestamp and class info
        fig.text(0.5, 0.02,
                f"Generated on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} | Classes: {', '.join(self.class_names)}",
                ha='center', fontsize=10)

        # Save summary report
        summary_path = os.path.join(self.results_dir, "summary_report.png")
        plt.savefig(summary_path, bbox_inches='tight', dpi=150)
        plt.close()

        logger.info(f"Summary report saved to {summary_path}")

In [ ]:
# Example usage
if __name__ == "__main__":
    # Update these paths with your actual paths
    MODEL_PATH = "/content/drive/MyDrive/Graduation Project/saved_models/optimized_model_v4/final_model.keras"
    DATA_DIR = "/content/drive/MyDrive/Graduation Project/disease_data"

    tester = ModelTester(
        model_path=MODEL_PATH,
        data_dir=DATA_DIR,
        img_size=(384, 384),
        batch_size=32
    )

    tester.run_full_evaluation()